In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import os
MNIST_PATH = os.path.join("datasets", "mnist")

In [5]:
import pandas as pd

def load_mnist_data(filename, mnist_path = MNIST_PATH):
    csv_path = os.path.join(mnist_path, filename)
    return pd.read_csv(csv_path)
    

In [6]:
train_data = load_mnist_data('train.csv')
test_data = load_mnist_data('test.csv')

In [7]:
X_test = test_data[:]
X_test.shape

(28000, 784)

In [12]:
X_train_full = train_data[:]
X_train_full.shape

(42000, 785)

In [13]:
y_train_full = train_data["label"]

In [14]:
y_train_full.shape

(42000,)

In [15]:
X_train_full = train_data.drop("label", axis = 1)
X_train_full.shape

(42000, 784)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [18]:
X_train.shape

(31500, 784)

In [19]:
X_valid.shape

(10500, 784)

In [20]:
X_test.shape

(28000, 784)

In [21]:
y_train.shape

(31500,)

In [22]:
y_valid.shape

(10500,)

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [24]:
y_train = keras.utils.to_categorical(y_train)
y_valid = keras.utils.to_categorical(y_valid)

In [31]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

### CONV MODEL

In [30]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", input_shape = [28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation = "softmax")
])

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = SGD, metrics = ["accuracy"])
history = model.fit(X_train, y_train, epochs = 10, validation_data = [X_valid, y_valid])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import numpy as np
y_pred = pd.DataFrame(y_pred.argmax(axis = 1))
y_pred.index.name = 'ImageId'
y_pred = y_pred.rename(columns = {0: 'Label'}).reset_index()
y_pred['ImageId'] = y_pred['ImageId'] + 1

y_pred.head()

In [ ]:
y_pred.to_csv('mnist_submission.csv', index = False)

### Using the above model (model), I am able to achieve score of 99.085, which stands at 1054 position on Leaderboard

### consider second model

In [ ]:
model1 = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", input_shape = [28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation = "softmax")
])

In [ ]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
model1.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
epochs = 35
batch_size = 86

In [ ]:
# With data augmentation to prevent overfitting

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
# Fit the model
history = model1.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_valid,y_valid),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

### Using the second model (model1), I am able to achieve score of 99.514, which stands at 480 position on Leaderboard

### Consider the third model

In [ ]:
model2 = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2, 2),
keras.layers.Dropout(0.2),
keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2, 2),
keras.layers.Dropout(0.3),
keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2, 2),
keras.layers.Dropout(0.4),
keras.layers.Flatten(),
keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation='softmax')
])

In [ ]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
model2.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
epochs = 50
batch_size = 86

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
# Fit the model
history = model2.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_valid,y_valid),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
y_pred = pd.DataFrame(y_pred.argmax(axis = 1))
y_pred.index.name = 'ImageId'
y_pred = y_pred.rename(columns = {0: 'Label'}).reset_index()
y_pred['ImageId'] = y_pred['ImageId'] + 1

y_pred.head()

In [ ]:
y_pred.to_csv('mnist_submission_model2.csv', index = False)

### Using the third model (model2), I am able to achieve score of 99.628, which stands at 327 position on Leaderboard